In [ ]:
dataset_path = '/content/drive/MyDrive/Emotions'


In [ ]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
!pip install resampy librosa numpy scikit-learn keras


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install resampy


In [ ]:
# Function to extract features from each audio file
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

In [ ]:
def load_data(dataset_path):
    features = []
    labels = []
    for emotion_folder in os.listdir(dataset_path):
        emotion_folder_path = os.path.join(dataset_path, emotion_folder)
        if os.path.isdir(emotion_folder_path):
            for file in os.listdir(emotion_folder_path):
                file_path = os.path.join(emotion_folder_path, file)
                if os.path.isfile(file_path):  # Ensure the path is a file
                    features.append(extract_features(file_path))
                    labels.append(emotion_folder)  # The folder name is the label
    return np.array(features), np.array(labels)


In [ ]:
!pip install resampy
X, y = load_data(dataset_path)

# Encode labels
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), verbose=1)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 20ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


4/4 [==============================] - 0s 2ms/step
Test Accuracy: 100.00%


In [ ]:
# Save the model in the recommended Keras format
model.save("speech_emotion_recognition_model.keras")


In [ ]:
#new_audio_file = '/content/03-01-06-01-01-01-01.wav' #Fearful
#new_audio_file = '/content/03-01-07-01-01-01-01.wav' #Disgusted
#new_audio_file = '/content/YAF_rag_angry.wav'
new_audio_file = '/content/03-01-03-01-01-01-01.wav' #Happy
features = extract_features(new_audio_file)
features = np.expand_dims(features, axis=0)
prediction = model.predict(features)
predicted_label = le.inverse_transform([np.argmax(prediction)])
print(f"Predicted Emotion: {predicted_label[0]}")


1/1 [==============================] - 0s 33ms/step
Predicted Emotion: Disgusted
